<a href="https://colab.research.google.com/github/JohnPaul-del/med_project/blob/main/Diplom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Установка необходимых модулей для работы.
"""

!pip install tensorflow==2.9.1          # Даунгрейд tensorflow необходим по причине того, что в новых версиях не работает сохранение модели архитектуры EfficientNet
!pip install pydicom                    # Бибилиотека работы с dcm файлами
!pip install tensorflow_addons          # Расширение возможностей tensorflow.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.9.1


In [ ]:
"""
Монтируем диск на котором находятся датасет и csv-файл с метками
"""

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Распаковываем архив с png изображениями снимков
"""

!tar -xvf '/content/drive/MyDrive/Colab Notebooks/Diploma/tmp.tar.gz'

Выходные данные были обрезаны до нескольких последних строк (5000).
tmp/ID_109a3a126.png
tmp/ID_7701245ef.png
tmp/ID_79eac961f.png
tmp/ID_006e59ac5.png
tmp/ID_323b50a10.png
tmp/ID_097527065.png
tmp/ID_75327dba0.png
tmp/ID_3c56a23fa.png
tmp/ID_082dd76f7.png
tmp/ID_0e4a787a3.png
tmp/ID_723e2b711.png
tmp/ID_44940ecc7.png
tmp/ID_32e51225b.png
tmp/ID_051b9ea25.png
tmp/ID_716bc99dd.png
tmp/ID_82feee45c.png
tmp/ID_7bec9a013.png
tmp/ID_58d3cb4e1.png
tmp/ID_18077598d.png
tmp/ID_058ae2630.png
tmp/ID_7019885d4.png
tmp/ID_3aac68039.png
tmp/ID_286764a26.png
tmp/ID_743e76a4c.png
tmp/ID_258d9c749.png
tmp/ID_63c5c573b.png
tmp/ID_60749d80a.png
tmp/ID_6b1223172.png
tmp/ID_497ecfcd5.png
tmp/ID_6672398c0.png
tmp/ID_747dbfc2e.png
tmp/ID_1c0302f61.png
tmp/ID_6bc71352e.png
tmp/ID_2993a72da.png
tmp/ID_1ab1011fa.png
tmp/ID_0aa6fe796.png
tmp/ID_4c7cc146c.png
tmp/ID_2d32f5a31.png
tmp/ID_711487b8d.png
tmp/ID_1a6b811b1.png
tmp/ID_0eecd6686.png
tmp/ID_049ff70f5.png
tmp/ID_5484addd5.png
tmp/ID_3c39c88a6.png
tmp/ID_2

In [ ]:
"""
Необходимые для работы импорты
"""

import os
from random import randint
import itertools
from glob import glob
from pathlib import Path
from tqdm import tqdm

import cv2
import pandas as pd
import numpy as np
import pydicom
import matplotlib.pylab as plt
from PIL import Image

from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow.keras import initializers, regularizers, constraints
from keras import backend as K
from tensorflow.keras.activations import elu
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout, Layer, InputSpec
from keras.losses import binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.metrics import IoU

from tensorflow_addons.optimizers import RectifiedAdam as RAdam

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're us

In [ ]:
"""
Константы
"""

TRAIN_DIR = 'rsna/train/'
TEST_DIR = 'rsna/stage_2_test'
SIZE = 224
BATCH_SIZE = 32
IMG_DIR = '/content/tmp/'


In [ ]:
"""
Классы
"""

class NormalizationLayer(Layer):
    """
    Слой групповой номрализации для замены слоев BatchNormalization.

    """

    def __init__(self,
                 groups=32,
                 axis=-1,
                 epsilon=1e-4,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):

        super(NormalizationLayer, self).__init__(**kwargs)
        self.support_mask = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = initializers.get(beta_initializer)
        self.gamma_initializer = initializers.get(gamma_initializer)
        self.beta_regularizer = regularizers.get(beta_regularizer)
        self.gamma_regularizer = regularizers.get(gamma_regularizer)
        self.beta_constraint = constraints.get(beta_constraint)
        self.gamma_constraint = constraints.get(gamma_constraint)

    def build(self, input_shape):

        dim = input_shape[self.axis]

        if dim is None:
            raise ValueError('Should have a dimension but recieved an input with shape')
        if dim < self.groups:
            raise ValueError(f'Groups can`t be more than channels.{str(self.group)} < {str(dim)}')
        if dim % self.groups != 0:
            raise ValueError('Error: Groups must be multiple of num channels')

        self.input_spec = InputSpec(ndim=len(input_shape),
                                             axes={self.axis: dim})
        shape = (dim, )

        if self.scale:
            self.gamma = self.add_weight(shape=shape,
                                         name='Gamma',
                                         initializer=self.gamma_initializer,
                                         regularizer=self.gamma_regularizer,
                                         constraint=self.beta_constraint)
        else:
            self.gamma = None

        if self.center:
            self.beta = self.add_weight(shape=shape,
                                        name='Beta',
                                        initializer=self.beta_initializer,
                                        regularizer=self.beta_regularizer,
                                        constraint=self.beta_constaint)
        else:
            self.beta = None

        self.built = True

    def call(self, inputs, **kwargs):
        input_shape = K.int_shape(inputs)
        tensor_input_shape = K.shape(inputs)

        reduction_axes = list(range(input_shape))
        del reduction_axes[self.axis]

        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(1, self.groups)

        reshape_groups = K.shape(inputs)
        group_axes = [reshape_groups[i] for i in range(len(input_shape))]

        group_shape = [group_axes[0], self.groups] + group_axes[2:]
        group_shape = K.stack(group_shape)
        inputs = K.reshape(inputs, group_shape)

        group_reduction_axes = list(range(len(group_axes)))
        group_reduction_axes = group_reduction_axes[2:]

        mean_val = K.mean(inputs, axis=group_reduction_axes, keepdims=True)
        variance = K.var(inputs, axis=group_reduction_axes, keepdims=True)

        inputs = (inputs - mean_val) / (K.sqrt(variance + self.epsilon))

        inputs = K.reshape(inputs, group_shape)
        outputs = inputs

        if self.scale:
            broadcast_gamma = K.reshape(self.gamma, broadcast_shape)
            outputs = outputs * broadcast_gamma

        if self.center:
            broadcast_beta = K.reshape(self.beta, broadcast_shape)
            outputs = outputs + broadcast_beta

        outputs = K.reshape(outputs, tensor_input_shape)

        return outputs

    def get_config(self):

        config = {
            'groups': self.groups,
            'axis': self.axis,
            'epsilon': self.epsilon,
            'center': self.center,
            'scale': self.scale,
            'beta_initializer': initializers.serialize(self.beta_initializer),
            'gamma_initializer': initializers.serialize(self.gamma_initializer),
            'beta_regularizer': regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer': regularizers.serialize(self.gamma_regularizer),
            'beta_constaint': constraints.serialize(self.beta_constraint),
            'gamma_constaint': constraints.serialize(self.gamma_constraint),
        }

        base_config = super(NormalizationLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def comp_output_shape(self, input_shape):
        return input_shape

In [ ]:
col_x = 'filename'
col_y = ['any', 'epidural', 'intraparenchymal',
               'intraventricular', 'subarachnoid', 'subdural']

"""
Функции
"""

def window_image(img, center, width, intercept, slope, rescale=True):

    """
    Rescale dicom objects. Create window
    :param:
    img - Dicom object
    center - center coordinates
    width - width
    intercept - intercept
    slope  - slope
    rescale - True or false
    :return:
    image object
    """

    img = (img * slope + intercept)
    img_min = center - width // 2
    img_max = center + width // 2
    img[img < img_min] = img_min
    img[img > img_max] = img_max

    if rescale:
        img = (img - img_min) / (img_max - img_min)

    return img


def get_dicom_fields(dcm_obj):
    """
    Get data from from DICOM object
    :param: dcm_obj
    :return: value of field

    """

    if type(dcm_obj) == pydicom.multival.MultiValue:
        return int(dcm_obj[0])
    else:
        return int(dcm_obj)


def get_sizes(dcm_obj):
    """
    Get values of image size
    :param: dcm_obj
    :return: image`s size
    """

    dicom_fields = [dcm_obj[('0028','1050')].value,
                    dcm_obj[('0028','1051')].value,
                    dcm_obj[('0028','1052')].value,
                    dcm_obj[('0028','1053')].value]

    return [get_dicom_fields(el) for el in dicom_fields]


def resize_dcm(dcm_files, load_dir):

    """
    Resize and convert DICOM objcts to PNG
    """

    save_dir = 'rsna/tmp/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    for file_name in tqdm(dcm_files):
        try:
            path = load_dir + file_name
            new_path = save_dir + file_name.replace('.dcm', '.png')
            dcm = pydicom.dcmread(path)

            window_center, window_width, intercept, slope = get_sizes(dcm)
            img = dcm.pixel_array
            img = window_image(img, window_center, window_width, intercept, slope)
            resized = cv2.resize(img, (224, 224))
            result = cv2.imwrite(new_path, resized)

        except ValueError:
            continue


def make_model(effnet):

    """
    Create model with EfficientNet layer
    :param: EfficientNet model parameters
    :return: Model with output layers
    """

    model = Sequential()
    model.add(effnet)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='sigmoid'))
    model.compile(
        loss=binary_crossentropy,
        optimizer=RAdam(warmup_proportion=0.1, learning_rate=0.00005),
        metrics=IoU(len(col_y), [0])
    )

    print(model.summary())

    return model


def preprocess_img(image, size, sigma_x=10):
    """
    Read image, resize and add noise
    """

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (size, size))
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), sigma_x), -4, 128)

    return image


def create_datagen():
    """
    create ImageDataGenerator with split 15%

    """
    return ImageDataGenerator(validation_split=0.15)


def create_test_gen(df, directory, x_col, size, batch_size):
    """
    Create testing ImageDataGenerator
    :parameters:
    df - Dataframe with labels
    directory - Dir with images
    x_col - col in dataframe with image names
    size - Size of image
    batch_size - batch Size
    """
    return ImageDataGenerator().flow_from_dataframe(
        df,
        directory=directory,
        x_col=x_col,
        class_mode=None,
        target_size=(size, size),
        batch_size=batch_size,
        shuffle=True
    )


def create_flow(datagen, subset, prep_df, directory, x_col, y_col, size, batch_size):
    """'target_class_ids'
    Create testing ImageDataGenerator
    :parameters:
    datageb - ImageDataGenerator
    subset - Train or validation
    prep_df - Dataframe with labels
    directory - Dir with images
    x_col - col in dataframe with image names
    y_col - cols with labels of type
    size - Size of image
    batch_size - batch Size

    :return:
    ImageDataGenerator
    """

    return datagen.flow_from_dataframe(
        prep_df,
        directory=directory,
        x_col=x_col,
        y_col=y_col,
        class_mode='other',
        target_size=(size, size),
        batch_size=batch_size,
        preprocessing_function=preprocess_img,
        rotation_range=360,
        horizontal_flip=True,
        validation_split=0.15,
        rescale=1 / 128.,
        subset=subset
    )


def bse_dice_loss(y_true, y_pred, smooth=1e-6):

  y_true = K.cast(y_true, 'float32')
  y_pred = K.cast(y_pred, 'float32')

  inputs = K.flatten(y_pred)
  targets = K.flatten(y_true)

  binary_loss = binary_crossentropy(targets, inputs)
  intersection = inputs * targets

  dice_loss = 1 - (2 * intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)

  dice_bce = binary_loss + dice_loss

  return dice_bce

In [ ]:
"""
Загрузка и обработка датафреймов
"""

df = pd.read_csv('rsna/train.csv')
df['filename'] = df['ID'].apply(lambda el: "ID_" + el.split('_')[1] + '.png')    # Получаем ID файла и добавляем раширение png
df['type'] = df['ID'].apply(lambda el: el.split('_')[2])                         # Получаем типы кровоизлияния в отдельный столбец

sample_df = df[df.filename.apply(lambda x: x.replace('.png', '.dcm')).isin(sample_files)]   # Отсавляем в датафрейме только те объекты, которые есть в каталоге

pivot_df = sample_df[['Label', 'filename', 'type']].drop_duplicates().pivot(
    index='filename',                                                            # Сводная таблица для выноса каждого типа в отдельный столбец. Поулчаем своеобразные OHE
    columns='type',
    values='Label').reset_index()

print(pivot_df.shape)

sample_df.to_csv('rsna/sample_df.csv')
pivot_df.to_csv('rsna/clear_df.csv')



In [ ]:
sample_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Diploma/sample_df.csv')
pivot_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Diploma/clear_df.csv')

In [ ]:
sample_df.head()

,Unnamed: 0,ID,Label,filename,type
0,6,ID_38fd7baa0_epidural,0,ID_38fd7baa0.png,epidural
1,7,ID_38fd7baa0_intraparenchymal,0,ID_38fd7baa0.png,intraparenchymal
2,8,ID_38fd7baa0_intraventricular,0,ID_38fd7baa0.png,intraventricular
3,9,ID_38fd7baa0_subarachnoid,0,ID_38fd7baa0.png,subarachnoid
4,10,ID_38fd7baa0_subdural,0,ID_38fd7baa0.png,subdural


In [ ]:
pivot_df.head()

,Unnamed: 0,filename,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,0,ID_000039fa0.png,0,0,0,0,0,0
1,1,ID_00005679d.png,0,0,0,0,0,0
2,2,ID_0000950d7.png,0,0,0,0,0,0
3,3,ID_0000aee4b.png,0,0,0,0,0,0
4,4,ID_0000ca2f6.png,0,0,0,0,0,0


In [ ]:

# Определяем колонки
col_x = 'filename'
col_y = ['any', 'epidural', 'intraparenchymal',
               'intraventricular', 'subarachnoid', 'subdural']

# Рандомно берем изображения для тренировочной и валидационной выборок
np.random.seed(2000)
sample_files = np.random.choice(os.listdir('/content/tmp/'), 400000)

# Создаем генератор для тренировочной выборки
data_generator = create_datagen()
train_gen = create_flow(
    datagen=data_generator,
    subset='training',
    prep_df=pivot_df,
    directory=IMG_DIR,
    x_col=col_x,
    y_col=col_y,
    size=SIZE,
    batch_size=BATCH_SIZE
)

# Генератор для валидационной выборки
val_gen = create_flow(
    datagen=data_generator,
    subset='validation',
    prep_df=pivot_df,
    directory=IMG_DIR,
    x_col=col_x,
    y_col=col_y,
    size=SIZE,
    batch_size=BATCH_SIZE
)

Found 174617 validated image filenames.
Found 30814 validated image filenames.


In [ ]:
"""
Создаем EfficientNetB4 и загружаем предобученные веса Imagenet
После загрузки, заменяем слои Батч нормализации на слой групповой нормализации.
"""

effnet = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

for el, layer in enumerate(effnet.layers):
    if 'batch_normalization' in layer.name:
        effnet.layers[el] = NormalizationLayer(groups=32, axis=-1, epsilon=0.0001)

model = make_model(effnet)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb4 (Functional)  (None, 7, 7, 1792)       17673823  
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1792)              0         
                                                                 
 dense (Dense)               (None, 6)                 10758     
                                                                 
Total params: 17,684,581
Trainable params: 17,559,374
Non-trainable params: 125,207
_________________________________________________________________
None


In [ ]:
"""
Создаем чекпоинт, И обучаем модель
"""

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, min_lr=0.0001)

ch_point = ModelCheckpoint(
    '/content/drive/MyDrive/Colab Notebooks/Diploma/rsna_effnetb5iou_dip.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto'
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    callbacks=[reduce_lr, ch_point],
    epochs=1,
)


5457/5457 [==============================] - 4476s 811ms/step - loss: 0.1551 - io_u: 0.9412 - val_loss: 0.1224 - val_io_u: 0.9394 - lr: 5.0000e-05


In [ ]:
history.history

{'loss': [0.1550508737564087],
 'io_u': [0.9411913156509399],
 'val_loss': [0.12242568284273148],
 'val_io_u': [0.9394052624702454],
 'lr': [5e-05]}

In [ ]:
results = (history.history['accuracy'], history.history['val_accuracy'], history.history['loss'], history.history['val_loss'])

In [ ]:
# Созраняем модель для последующего использования

model.save('/content/drive/MyDrive/Colab Notebooks/Diploma/rsna_mod_tf2_effnet_b5.h5')
